<a href="https://colab.research.google.com/github/Eyeteebe/MLSalon/blob/master/resources/word2vec_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import nltk
nltk.download('punkt')
import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from gensim.models import FastText
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('ted_en-20160408.xml')

targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

# Getting contents of <content> tag from the xml file
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# Removing "Sound-effect labels" using regular expression (i.e. (Audio), (Laughter))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# Tokenising the sentence to process it by using NLTK library
sent_text=sent_tokenize(content_text)

# Removing punctuations and changing all characters to lower case
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# Tokenising each sentence to process individual word
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text]

# Prints only 10 (tokenised) sentences
print(sentences[:10])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doing', 'exactly', 'the', 'same']]


In [ ]:
def splite_words(formula):
  word_list_n = []
  word_list_p = []
  single_word = ""
  word_len = 0
  word_group = 1
  for bit in formula:
    if bit == '-':
      if word_group == 0:
        word_list_n.append(single_word[:word_len])
      else:
        word_list_p.append(single_word[:word_len])
      word_len = 0
      single_word = ""
      word_group = 0
    elif bit == '+':
      if word_group == 0:
        word_list_n.append(single_word[:word_len])
      else:
        word_list_p.append(single_word[:word_len])
      word_len = 0
      single_word = ""
      word_group = 1
    else:
      single_word += bit
      word_len += 1

  if word_group == 0:
    word_list_n.append(single_word[:word_len])
  else:
    word_list_p.append(single_word[:word_len])

  return word_list_n, word_list_p

In [ ]:
def get_4_models(sentences):
  wv_cbow_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4, sg=0)
  wv_sg_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4, sg=1)
  ft_cbow_model = FastText(sentences, vector_size=100, window=5, min_count=5, workers=4, sg=0)
  ft_sg_model = FastText(sentences, vector_size=100, window=5, min_count=5, workers=4, sg=1)
  return wv_cbow_model, wv_sg_model, ft_cbow_model, ft_sg_model


wv_cbow_model, wv_sg_model, ft_cbow_model, ft_sg_model = get_4_models(sentences)

In [ ]:
#@title Word Algebra Calculator

#@markdown Please select the model and formula to calculate the word algebra

# Get the input
Formula = 'Family - parents' #@param {type: "string"}
Formula = re.sub(" ", "", Formula.lower())
word_list_p, word_list_n = splite_words(Formula)
#print(word_list_n, word_list_p)
Model = 'Word2Vec' #@param ['Word2Vec', 'FastText']
Architecture = "Skip Gram" #@param ["CBOW", "Skip Gram"]
if Model == "Word2Vec":
  if Architecture == "CBOW":
    result = wv_cbow_model.wv.most_similar(positive=word_list_p, negative=word_list_n, topn=1)
  else:
    result = wv_sg_model.wv.most_similar(positive=word_list_p, negative=word_list_n, topn=1)
else:
  if Architecture == "CBOW":
    result = ft_cbow_model.wv.most_similar(positive=word_list_p, negative=word_list_n, topn=1)
  else:
    result = ft_sg_model.wv.most_similar(positive=word_list_p, negative=word_list_n, topn=1)
print("The result is:", result[0][0])

#@markdown Now you can activate the Calculator by running this section

The result is: kids
